# Food Vision - Milestone

In [1]:
# Importing TF and checking the version
import tensorflow as tf

print(tf.__version__)

2.10.1


In [2]:
# Getting helper functions
from DanielBourke_HelperFunctions import create_tensorboard_callback, plot_loss_curves, compare_historys

### Using TensorFlow Datasets (TFDS)

In [3]:
# Getting TensorFlow Datasets
import tensorflow_datasets as tfds

In [4]:
# Listing all available datasets
datasets_list = tfds.list_builders()
#datasets_list

In [5]:
# Loading in the data
(train_data, test_data), ds_info = tfds.load(
    name="food101",
    split=["train", "validation"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

### Exploring TFDS Food101 dataset

In [6]:
# Features of Food101 dataset from TFDS
ds_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

In [7]:
# Get the class names
class_names = ds_info.features["label"].names
class_names[:10]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

In [8]:
# Taking one sample of train data
train_one_sample = train_data.take(1) # Samples are in format (image_tensor, label)
train_one_sample

<TakeDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [9]:
# Output info about our sample
for image, label in train_one_sample:
    print(f"""
    Image shape: {image.shape}
    Image datatype: {image.dtype}
    Target class: {label}
    Class name(str form): {class_names[label.numpy()]}
    """)


    Image shape: (382, 512, 3)
    Image datatype: <dtype: 'uint8'>
    Target class: 52
    Class name(str form): gyoza
    


In [10]:
# Sample image structure
image

<tf.Tensor: shape=(382, 512, 3), dtype=uint8, numpy=
array([[[207, 220, 237],
        [207, 220, 237],
        [207, 220, 236],
        ...,
        [ 10,  11,  13],
        [ 11,  12,  14],
        [ 12,  13,  15]],

       [[208, 219, 237],
        [207, 220, 236],
        [208, 220, 236],
        ...,
        [  9,  10,  12],
        [ 10,  11,  13],
        [ 11,  12,  14]],

       [[216, 225, 242],
        [213, 225, 239],
        [214, 223, 238],
        ...,
        [ 11,  12,  14],
        [ 12,  13,  15],
        [ 12,  13,  15]],

       ...,

       [[ 33,  10,  16],
        [ 34,  11,  17],
        [ 33,  12,  17],
        ...,
        [ 10,   6,   5],
        [ 13,   7,   7],
        [ 15,   9,   9]],

       [[ 37,  12,  18],
        [ 36,  13,  19],
        [ 35,  15,  17],
        ...,
        [ 14,  10,   9],
        [ 19,  13,  13],
        [ 21,  15,  15]],

       [[ 39,  14,  20],
        [ 37,  14,  20],
        [ 36,  14,  17],
        ...,
        [ 19,  15,  1

In [11]:
# Checking image's min/max values
tf.reduce_min(image), tf.reduce_max(image)

(<tf.Tensor: shape=(), dtype=uint8, numpy=0>,
 <tf.Tensor: shape=(), dtype=uint8, numpy=255>)

### Plotting TFDS sample image

(-0.5, 511.5, 381.5, -0.5)

In [12]:
# Plotting an image tensor
import matplotlib.pyplot as plt

plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis(False)

### Building a preprocessing function

In [13]:
# Making a function
def preprocess_img(image, label, img_shape=224):
    # Resize the image
    image = tf.image.resize(image, [img_shape, img_shape])
    # image = image / 255. # Optional rescaling
    # Convert image from uint8 to float32
    return tf.cast(image, tf.float32), label

In [14]:
# Preporcessing sample tensor
preprocessed_img = preprocess_img(image, label)[0]
print(f"""
Image before preprocessing: {image[:2]}
Image shape: {image.shape}
Image data type: {image.dtype}
Image after preprocessing: {preprocessed_img[:2]}
Image shape: {preprocessed_img.shape}
Image data type: {preprocessed_img.dtype}
""")


Image before preprocessing: [[[207 220 237]
  [207 220 237]
  [207 220 236]
  ...
  [ 10  11  13]
  [ 11  12  14]
  [ 12  13  15]]

 [[208 219 237]
  [207 220 236]
  [208 220 236]
  ...
  [  9  10  12]
  [ 10  11  13]
  [ 11  12  14]]]
Image shape: (382, 512, 3)
Image data type: <dtype: 'uint8'>
Image after preprocessing: [[[207.12596  219.87404  236.77328 ]
  [206.75159  219.39891  235.39891 ]
  [206.33514  216.86098  232.36958 ]
  ...
  [ 10.433058  11.433058  13.433058]
  [  9.974804  10.974804  12.974804]
  [ 11.004499  12.004499  14.004499]]

 [[214.51498  225.25287  240.26627 ]
  [214.24043  223.17825  238.07462 ]
  [213.40561  221.28954  234.25638 ]
  ...
  [ 12.571472  13.571472  15.571472]
  [ 11.928558  12.928558  14.928558]
  [ 11.941964  12.941964  14.941964]]]
Image shape: (224, 224, 3)
Image data type: <dtype: 'float32'>



### Batch and prepare datasets

In [15]:
# Map preprocessing function to training (and parallelise it)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle training data and turn into batches
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map preprocessing function to test
test_data = test_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

In [16]:
train_data, train_data

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

### Create modelling callbacks

In [17]:
# Getting callback function from the helper package
from DanielBourke_HelperFunctions import create_tensorboard_callback

In [18]:
# Creating ModelCheckpoint callback to save model's progress
checkpoint_path = "model_checkpoints/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor="val_acc",
    save_best_only=True,
    save_weights_only=True,
    verbose=0 # Don't print anything
)

### Setup mixed precision policy

In [19]:
# Activating mixed precision
from tensorflow.keras import mixed_precision
# mixed_precision.set_global_policy("mixed_float16") # SETTING IT ON
mixed_precision.set_global_policy("float32") # SETTING IT OFF

In [20]:
# Checking global policy settings
mixed_precision.global_policy()

<Policy "float32">

### Building feature extraction model

In [21]:
# Creating a base model
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Creating functional model
inputs = layers.Input(shape=input_shape, name="input_layer")
# x = preprocessing.Rescaling(1/255.)(x)
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax")(x)
model = tf.keras.Model(inputs, outputs)

# Compiling the model
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [22]:
# Model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 101)               129381    
                                                                 
 activation (Activation)     (None, 101)               0         
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
_____________________________________________

In [23]:
# Checking data types at each layer level
for layer in model.layers:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_layer True float32 <Policy "float32">
efficientnetb0 False float32 <Policy "float32">
global_average_pooling2d True float32 <Policy "float32">
dense True float32 <Policy "float32">
activation True float32 <Policy "float32">


In [24]:
# Fitting the model
history1 = model.fit(
    train_data,
    epochs=3,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=int(0.15 * len(test_data)),
    callbacks=[create_tensorboard_callback(
        dir_name="training_logs",
        experiment_name="efficientb0"
    ),
    model_checkpoint]
)

Saving TensorBoard log files to: training_logs/efficientb0/20230106-094453


Epoch 1/3
2368/2368 [==============================] - ETA: 0s - loss: 1.7169 - accuracy: 0.5822WARNING:tensorflow:Can save best model only with val_acc available, skipping.


2368/2368 [==============================] - 4458s 2s/step - loss: 1.7169 - accuracy: 0.5822 - val_loss: 1.1166 - val_accuracy: 0.7005
Epoch 2/3
2368/2368 [==============================] - ETA: 0s - loss: 1.1992 - accuracy: 0.6901WARNING:tensorflow:Can save best model only with val_acc available, skipping.


2368/2368 [==============================] - 2363s 997ms/step - loss: 1.1992 - accuracy: 0.6901 - val_loss: 1.0388 - val_accuracy: 0.7246
Epoch 3/3
2368/2368 [==============================] - ETA: 0s - loss: 1.0529 - accuracy: 0.7235WARNING:tensorflow:Can save best model only with val_acc available, skipping.


2368/2368 [==============================] - 2643s 1s/step - loss: 1.0529 - accuracy: 0.7235 - val_loss: 0.9984 - val_accuracy: 0.7291


In [25]:
# Evaluating the model on the full dataset
results = model.evaluate(test_data)
results

790/790 [==============================] - 825s 1s/step - loss: 1.0025 - accuracy: 0.7264


[1.002529263496399, 0.7264158129692078]